In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install finance-datareader
!pip install neuralprophet[live]

     |████████████████████████████████| 55 kB 3.4 MB/s 
     |████████████████████████████████| 155 kB 37.9 MB/s 
  Attempting uninstall: holidays
    Found existing installation: holidays 0.10.5.2
    Uninstalling holidays-0.10.5.2:
      Successfully uninstalled holidays-0.10.5.2


In [3]:
import numpy as np
import pandas as pd
import gc
import math
import os.path
import time
import matplotlib.pyplot as plt
from datetime import timedelta, datetime
from dateutil import parser
from tqdm import tqdm
import copy
from fbprophet import Prophet

from neuralprophet import NeuralProphet
from neuralprophet import set_random_seed 

import warnings
import datetime
import random
import tensorflow as tf
warnings.filterwarnings("ignore")

# PUBLIC

In [4]:
path = '/content/drive/MyDrive/stock_2/'
list_name = 'stock_list.csv'
sample_name = 'sample_submission.csv'

stock_list = pd.read_csv(os.path.join(path,list_name))
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x : str(x).zfill(6))
stock_list

,종목명,종목코드,상장시장
0,삼성전자,005930,KOSPI
1,SK하이닉스,000660,KOSPI
2,NAVER,035420,KOSPI
3,카카오,035720,KOSPI
4,삼성바이오로직스,207940,KOSPI
...,...,...,...
365,맘스터치,220630,KOSDAQ
366,다날,064260,KOSDAQ
367,제이시스메디칼,287410,KOSDAQ
368,크리스에프앤씨,110790,KOSDAQ


In [5]:
import os
fileList = os.listdir('/content/drive/MyDrive/stock_2/train')

In [6]:
companyList = []
for file in fileList:
    companyName = file.split("_")[1]
    companyName = companyName.split('.')[0]
    if companyName != "csv":
        companyList.append(companyName)
companyList = sorted(companyList)
print(companyList)

['000060', '000080', '000100', '000120', '000150', '000240', '000250', '000270', '000660', '000670', '000720', '000810', '000880', '000990', '001230', '001440', '001450', '001740', '002380', '002790', '003000', '003090', '003380', '003410', '003490', '003670', '003800', '004000', '004020', '004170', '004370', '004490', '004800', '004990', '005250', '005290', '005300', '005380', '005385', '005387', '005490', '005830', '005850', '005930', '005935', '005940', '006260', '006280', '006360', '006400', '006650', '006730', '006800', '007070', '007310', '007390', '008560', '008770', '008930', '009150', '009240', '009540', '009830', '009900', '010060', '010120', '010130', '010620', '010780', '010950', '011000', '011070', '011170', '011200', '011210', '011780', '012330', '012450', '012510', '012750', '013120', '013890', '014680', '015750', '015760', '016360', '016380', '017670', '017800', '018260', '018880', '019170', '020150', '021240', '022100', '023530', '024110', '025900', '025980', '026960',

In [7]:
sample_submission = pd.read_csv(path + 'sample_submission.csv')
sample_submission = sample_submission[['Day']]
sample_submission['Day'] =pd.to_datetime(sample_submission['Day'])

In [8]:
for i in tqdm(sorted(companyList)):
    globals()['df_{}'.format(i)] = pd.read_csv('/content/drive/MyDrive/stock_2/train/stock_' + i + ".csv")
    locals()['df_{}'.format(i)] = locals()['df_{}'.format(i)][['Date', 'Close']]
    locals()['df_{}'.format(i)].rename(columns={'Date':'ds'}, inplace=True)
    locals()['df_{}'.format(i)].rename(columns={'Close':'y'}, inplace=True)

100%|██████████| 370/370 [01:44<00:00,  3.55it/s]


In [9]:
# model = NeuralProphet(
#     n_forecasts=5,
#         n_lags=370,
#         yearly_seasonality=False,
#         weekly_seasonality=False,
#         daily_seasonality=False,
#         batch_size=64,
#         epochs=100,
#         learning_rate=1.0,
# )

# metric = model.fit(df_000060, freq='D', valid_p=0.1, epochs=100)

# future = model.make_future_dataframe(df_000060, periods=7)
# forecast = model.predict(future)

In [11]:
for i in sorted(companyList):
    globals()['model_{}'.format(i)] = NeuralProphet(
        n_forecasts=5,
        n_lags=180,
        yearly_seasonality='auto',
        weekly_seasonality='auto',
        daily_seasonality='auto',
        batch_size=64,
        epochs=100,
        learning_rate=0.1,
        loss_func='mae',
        normalize='auto'
        )

    globals()['metric_{}'.format(i)] = locals()['model_{}'.format(i)].fit(locals()['df_{}'.format(i)],
    freq='D',
    valid_p=0.1,
    epochs=100)

    globals()['future_{}'.format(i)] = locals()['model_{}'.format(i)].make_future_dataframe(locals()['df_{}'.format(i)], periods=5)
    globals()['forecast_{}'.format(i)] = locals()['model_{}'.format(i)].predict(locals()['future_{}'.format(i)])

INFO - (NP.forecaster._handle_missing_data) - 95 missing dates added.
INFO - (NP.forecaster._handle_missing_data) - 95 NaN values in column y were auto-imputed.
INFO - (NP.utils.set_auto_seasonalities) - Disabling yearly seasonality. Run NeuralProphet with yearly_seasonality=True to override this.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
Epoch[100/100]: 100%|██████████| 100/100 [00:01<00:00, 60.01it/s, L1Loss=0.0409, MAE=664, MSE=7.85e+5, RegLoss=0]
INFO - (NP.forecaster._handle_missing_data) - 87 missing dates added.
INFO - (NP.forecaster._handle_missing_data) - 87 NaN values in column y were auto-imputed.
INFO - (NP.forecaster._handle_missing_data) - 95 missing dates added.
INFO - (NP.forecaster._handle_missing_data) - 95 NaN values in column y were auto-imputed.
INFO - (NP.utils.set_auto_seasonalities) - Disabling yearly seasonality. Run NeuralProphet with yearly_seasonality=True to overri

In [12]:
forecast_000060

,ds,y,yhat1,residual1,yhat2,residual2,yhat3,residual3,yhat4,residual4,yhat5,residual5,ar1,ar2,ar3,ar4,ar5,trend,season_weekly
0,2021-02-05,14500.000000,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,2021-02-06,14616.666667,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,2021-02-07,14733.333333,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,2021-02-08,14850.000000,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,2021-02-09,14900.000000,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,2021-10-30,NaN,28294.3,NaN,28282.7,NaN,30474.9,NaN,30556.3,NaN,30195.9,NaN,9326.54,9314.99,11507.2,11588.6,11228.2,18974.4,-6.66745
268,2021-10-31,NaN,None,NaN,28608.5,NaN,31782.9,NaN,30812,NaN,29961.8,NaN,None,9632.47,12806.8,11836,10985.7,18988.7,-12.6685
269,2021-11-01,NaN,None,NaN,None,NaN,31654.1,NaN,31361.1,NaN,29474.5,NaN,None,None,12846.9,12553.9,10667.3,19003,-195.787
270,2021-11-02,NaN,None,NaN,None,NaN,None,NaN,31862.6,NaN,30597.9,NaN,None,None,None,12869.6,11604.8,19017.3,-24.3255


In [13]:
for i in sorted(companyList):
    globals()['list_{}'.format(i)] = []
    locals()['list_{}'.format(i)].append(locals()['forecast_{}'.format(i)]['yhat1'][267])
    locals()['list_{}'.format(i)].append(locals()['forecast_{}'.format(i)]['yhat2'][268])
    locals()['list_{}'.format(i)].append(locals()['forecast_{}'.format(i)]['yhat3'][269])
    locals()['list_{}'.format(i)].append(locals()['forecast_{}'.format(i)]['yhat4'][270])
    locals()['list_{}'.format(i)].append(locals()['forecast_{}'.format(i)]['yhat5'][271])

In [14]:
sample_submission = pd.read_csv(path + 'sample_submission.csv')
sub_public = sample_submission[0:5]
sub_private = sample_submission[5:10]

In [15]:
for i in sorted(companyList):
    sub_public[i] = locals()['list_{}'.format(i)]

In [16]:
import os
fileList = os.listdir('/content/drive/MyDrive/stock_2/test')

In [17]:
companyList = []
for file in fileList:
    companyName = file.split("_")[1]
    companyName = companyName.split('.')[0]
    if companyName != "csv":
        companyList.append(companyName)
companyList = sorted(companyList)
print(companyList)

['000060', '000080', '000100', '000120', '000150', '000240', '000250', '000270', '000660', '000670', '000720', '000810', '000880', '000990', '001230', '001440', '001450', '001740', '002380', '002790', '003000', '003090', '003380', '003410', '003490', '003670', '003800', '004000', '004020', '004170', '004370', '004490', '004800', '004990', '005250', '005290', '005300', '005380', '005385', '005387', '005490', '005830', '005850', '005930', '005935', '005940', '006260', '006280', '006360', '006400', '006650', '006730', '006800', '007070', '007310', '007390', '008560', '008770', '008930', '009150', '009240', '009540', '009830', '009900', '010060', '010120', '010130', '010620', '010780', '010950', '011000', '011070', '011170', '011200', '011210', '011780', '012330', '012450', '012510', '012750', '013120', '013890', '014680', '015750', '015760', '016360', '016380', '017670', '017800', '018260', '018880', '019170', '020150', '021240', '022100', '023530', '024110', '025900', '025980', '026960',

In [18]:
for i in tqdm(sorted(companyList)):
    globals()['df_{}'.format(i)] = pd.read_csv('/content/drive/MyDrive/stock_2/test/stock_' + i + ".csv")
    locals()['df_{}'.format(i)] = locals()['df_{}'.format(i)][['Date', 'Close']]
    locals()['df_{}'.format(i)].rename(columns={'Date':'ds'}, inplace=True)
    locals()['df_{}'.format(i)].rename(columns={'Close':'y'}, inplace=True)

100%|██████████| 370/370 [01:14<00:00,  5.00it/s]


In [19]:
for i in sorted(companyList):
    globals()['model_{}'.format(i)] = NeuralProphet(
        n_forecasts=5,
        n_lags=120,
        yearly_seasonality='auto',
        weekly_seasonality='auto',
        daily_seasonality='auto',
        batch_size=64,
        epochs=100,
        learning_rate=0.1,
        loss_func='mae',
        normalize='auto'
        )

    globals()['metric_{}'.format(i)] = locals()['model_{}'.format(i)].fit(locals()['df_{}'.format(i)],
    freq='D',
    valid_p=0.1,
    epochs=100)

    globals()['future_{}'.format(i)] = locals()['model_{}'.format(i)].make_future_dataframe(locals()['df_{}'.format(i)], periods=5)
    globals()['forecast_{}'.format(i)] = locals()['model_{}'.format(i)].predict(locals()['future_{}'.format(i)])

INFO - (NP.forecaster._handle_missing_data) - 103 missing dates added.
INFO - (NP.forecaster._handle_missing_data) - 103 NaN values in column y were auto-imputed.
INFO - (NP.utils.set_auto_seasonalities) - Disabling yearly seasonality. Run NeuralProphet with yearly_seasonality=True to override this.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
Epoch[100/100]: 100%|██████████| 100/100 [00:02<00:00, 42.08it/s, L1Loss=0.0461, MAE=749, MSE=9.79e+5, RegLoss=0]
INFO - (NP.forecaster._handle_missing_data) - 56 missing dates added.
INFO - (NP.forecaster._handle_missing_data) - 56 NaN values in column y were auto-imputed.
INFO - (NP.forecaster._handle_missing_data) - 103 missing dates added.
INFO - (NP.forecaster._handle_missing_data) - 103 NaN values in column y were auto-imputed.
INFO - (NP.utils.set_auto_seasonalities) - Disabling yearly seasonality. Run NeuralProphet with yearly_seasonality=True to ov

In [20]:
forecast_000060

,ds,y,yhat1,residual1,yhat2,residual2,yhat3,residual3,yhat4,residual4,yhat5,residual5,ar1,ar2,ar3,ar4,ar5,trend,season_weekly
0,2021-06-04,17850.000000,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,2021-06-05,17783.333333,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,2021-06-06,17716.666667,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,2021-06-07,17650.000000,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,2021-06-08,17650.000000,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,2021-11-27,NaN,29153.1,NaN,30627.8,NaN,30296.1,NaN,32936.8,NaN,30939.3,NaN,10187.9,11662.7,11330.9,13971.6,11974.1,18851.4,113.704
177,2021-11-28,NaN,None,NaN,32246.2,NaN,30888.5,NaN,32441.9,NaN,30346.8,NaN,None,13062,11704.2,13257.7,11162.5,18855.4,328.842
178,2021-11-29,NaN,None,NaN,None,NaN,30804.2,NaN,30911.4,NaN,31096.8,NaN,None,None,11832.2,11939.4,12124.7,18859.4,112.651
179,2021-11-30,NaN,None,NaN,None,NaN,None,NaN,32317.2,NaN,30964.5,NaN,None,None,None,13446.3,12093.6,18863.4,7.56772


In [21]:
for i in sorted(companyList):
    globals()['list_{}'.format(i)] = []
    locals()['list_{}'.format(i)].append(locals()['forecast_{}'.format(i)]['yhat1'][176])
    locals()['list_{}'.format(i)].append(locals()['forecast_{}'.format(i)]['yhat2'][177])
    locals()['list_{}'.format(i)].append(locals()['forecast_{}'.format(i)]['yhat3'][178])
    locals()['list_{}'.format(i)].append(locals()['forecast_{}'.format(i)]['yhat4'][179])
    locals()['list_{}'.format(i)].append(locals()['forecast_{}'.format(i)]['yhat5'][180])

In [22]:
for i in sorted(companyList):
    sub_private[i] = locals()['list_{}'.format(i)]

In [23]:
sub_total = pd.concat([sub_public, sub_private])

In [24]:
sub_total.to_csv(path + 'neural_prophet_lag_180.csv', index=False)